In [ ]:
import csv
import os.path
from os import path
import re
import random
import requests
import json
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib
import ssl

In [ ]:
def select_players():
    """input player email addresses and returns valid emails as a list"""
    
    regex = '^(\w|\.|\_|\-)+[@](\w|\_|\-|\.)+[.]\w{2,3}$'
    players = input('enter player email addresses, separated by a space: ')
    players = players.split(' ')
    
    for i in players:
        if not re.search(regex, i):
            print("'%s' is not a valid email and will be removed" % i)
    return list(filter(lambda x: re.search(regex, x), players))

   
def load_categories(file):
    """read categories from csv"""
    
    categories = []
    if os.path.exists(file):
        with open(file) as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                categories.append(row)
        return categories
    else:
        print('\n invalid file \n')


def select_category():
    """input a category and returns the category as an integer"""
    
    try:
        selection = int(input("select a category: \n 0 - Food \n 1 - Movies \n 2 - Presidents \n 3 - Fairy Tales \n 4 - Countries \n 5 - Musical Instruments \n 6 - Film Genres \n 7 - Children's Books \n 8 - Transportation \n 9 - Under the Sea \n 10 - Historical Figures \n 11 - Inventions \n \n"))
        if selection in [0,1,2,3,4,5,6,7,8,9,10,11]:
            return selection
        else:
            print('\n invalid entry \n')
            select_category()
    except:
        print('\n invalid entry \n')
        select_category()
        
        
def randomize_teams(players): 
    """randomly select chameleon from list of players"""
    h = players.copy()
    c = h.pop(random.randint(0, len([h])))
    return h, c


def run_game(host, players):
    """takes the host email and players, selects a category, randomizes the teams, and send the emails to the players"""
    
    category = select_category()
    secret_word = random.choice(categories[category][1:])
    gif = json.loads(requests.get('https://api.giphy.com/v1/gifs/search?api_key=' + your_giphy_api + '&q=' + secret_word + '&limit=10&offset=0&rating=pg-13&lang=en').text)
    gif_url = gif['data'][random.randint(0, 9)]['images']['original']['url'] #range of random comes from limit in request
    humans, chameleon = randomize_teams(players)
    
    
    human_message = MIMEMultipart("alternative")
    human_message["Subject"] = "Virtual Chameleon!"
    human_html = """\
    <html>
      <body>
        <p>The category is <b>%s</b> and the secret word is <b>%s</b>.
        </p>
        <img src="%s">
      </body>
    </html>
    """ % (categories[category][0], secret_word, gif_url) #categories is a list of lists with first element the category name itself
    part1 = MIMEText(human_html, "html")
    human_message.attach(part1)
    

    chameleon_message = MIMEMultipart("alternative")
    chameleon_message["Subject"] = "Virtual Chameleon!"
    chameleon_html = """\
    <html>
      <body>
        <p>The category is <b>%s</b> and <b>you</b> are the chameleon - try to blend in!
        </p>
        <img src="https://thumbs.gfycat.com/ScornfulWateryAfricanmolesnake-max-1mb.gif">
      </body>
    </html>
    """ % (categories[category][0]) #categories is a list of lists with first element the category name itself
    part2 = MIMEText(chameleon_html, "html")
    chameleon_message.attach(part2)
    
    
    port = 465
    context = ssl.create_default_context()
    sender_email = host
    password = getpass()
    
    with smtplib.SMTP_SSL('smtp.gmail.com', port=port, context=context) as server:
        server.login(host, password=password)
        
        for i in humans: #must be sent one at a time
            server.sendmail(host, i, human_message.as_string())
            
        server.sendmail(host, chameleon, chameleon_message.as_string())

In [ ]:
players = select_players()
categories = load_categories('chameleon_categories.csv')

In [ ]:
run_game('your_email@gmail.com', players)